## Deliverable 2. Create a Customer Travel Destinations Map.

In [79]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import NumPy for Null checking
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [80]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Katsuura,JP,35.1333,140.3000,37.11,96,100,25.28,overcast clouds
1,1,Ixtapa,MX,20.7000,-105.2000,80.58,24,0,3.04,clear sky
2,2,Hambantota,LK,6.1241,81.1185,78.24,78,69,1.70,moderate rain
3,3,Husavik,IS,66.0449,-17.3389,28.31,83,81,13.09,broken clouds
4,4,Wanning,CN,18.8003,110.3967,69.60,93,25,5.64,scattered clouds


In [81]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("Enter minimum temperature desired for your trip: "))
max_temp = float(input("Enter maximum temperature desired for your trip: "))

In [82]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_temp_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_temp_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Katsuura,JP,35.1333,140.3000,37.11,96,100,25.28,overcast clouds
3,3,Husavik,IS,66.0449,-17.3389,28.31,83,81,13.09,broken clouds
19,19,Ketchikan,US,55.3422,-131.6461,39.18,87,100,5.75,overcast clouds
23,23,Nikolskoye,RU,59.7035,30.7861,36.88,90,95,10.67,light snow
32,32,Sitka,US,57.0531,-135.3300,37.29,75,100,14.97,mist
38,38,Kula,TR,38.5473,28.6498,35.01,65,0,3.36,clear sky
43,43,Otradnoye,RU,59.7775,30.8181,37.04,90,95,11.03,overcast clouds
48,48,Betsiamites,CA,48.9413,-68.6460,36.00,98,100,4.32,snow
59,59,Sorland,NO,67.6670,12.6934,35.67,61,100,29.51,light snow
62,62,Whitehorse,CA,60.7161,-135.0538,25.57,39,20,6.91,few clouds


In [83]:
# 4a. Determine if there are any empty rows.
preferred_cities_temp_df.count()


City_ID                122
City                   122
Country                122
Lat                    122
Lng                    122
Max Temp               122
Humidity               122
Cloudiness             122
Wind Speed             122
Current Description    122
dtype: int64

In [84]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# Use dropna at the end of the statement to drop the Null values
preferred_cities_df = preferred_cities_temp_df.dropna()

In [85]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name" and set it equal to Null so we can use the .dropna function later
hotel_df["Hotel Name"] = np.nan

In [86]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}


# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()

    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [87]:
# Check how many records have nulls in hotel name
hotel_df.count()


City                   122
Country                122
Max Temp               122
Current Description    122
Lat                    122
Lng                    122
Hotel Name             106
dtype: int64

In [88]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna(axis=0, subset=['Hotel Name'])
# Check the new results
hotel_df.count()

City                   106
Country                106
Max Temp               106
Current Description    106
Lat                    106
Lng                    106
Hotel Name             106
dtype: int64

In [89]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")



### Start of Marker Layer Map

In [90]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
# Variable is assigned to a multiline string using three quotes
# The <dl> tag is a description list (dl).
# The <dt> tag is a term or name in a description list that is nested under the <dl> tag.

info_box_template = """
<dl>
<dt><b>Hotel Name</b></dt><dt>{Hotel Name}</dt>
<dt><b>City</b></dt><dt>{City}</dt>
<dt><b>Country</b></dt><dt>{Country}</dt>
<dt><b>Current Weather</b></dt><dt>{Current Description} and {Max Temp} °F</dt>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
# In the list comprehension, info_box_template.format(**row) for index, row in hotel_df.iterrows(), 
# we iterate through each "row" of the hotel_df DataFrame and then format the info_box_template with 
# the data we set to populate the from each row. Remember, we are not using every row; we are only 
# using the rows defined in the info_box_template, which are Hotel Name, City, Country, and Current Weather.

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [94]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig



Figure(layout=FigureLayout(height='420px'))